In [4]:
from datasets import load_dataset

In [5]:
wnut = load_dataset("wnut_17")

Generating test split: 100%|██████████| 1287/1287 [00:00<00:00, 28063.79 examples/s]


In [9]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [14]:
from transformers import AutoTokenizer

In [15]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [17]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [18]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [19]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 1287/1287 [00:00<00:00, 20413.60 examples/s]


In [30]:
from transformers import DataCollatorForTokenClassification

In [31]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate
seqeval = evaluate.load("seqeval")

In [38]:
import numpy as np

In [39]:
labels = [label_list[i] for i in example[f"ner_tags"]]

In [41]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [42]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [43]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [44]:
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [46]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [47]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.279925,0.508167,0.259500,0.343558,0.938780
2,No log,0.268442,0.576214,0.318814,0.410501,0.941943


/mnt/c/Users/rebal/ner-tuning/ner/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=426, training_loss=0.2248642254323467, metrics={'train_runtime': 70.2623, 'train_samples_per_second': 96.609, 'train_steps_per_second': 6.063, 'total_flos': 91781128898820.0, 'train_loss': 0.2248642254323467, 'epoch': 2.0})

In [60]:
trainer.evaluate()

{'eval_loss': 0.2684415280818939,
 'eval_precision': 0.576214405360134,
 'eval_recall': 0.31881371640407785,
 'eval_f1': 0.41050119331742246,
 'eval_accuracy': 0.9419434825360181,
 'eval_runtime': 2.159,
 'eval_samples_per_second': 596.102,
 'eval_steps_per_second': 37.517,
 'epoch': 2.0}

In [54]:
trainer.save_model()

In [48]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [49]:
from transformers import pipeline

In [55]:
classifier = pipeline("ner", model="model")
classifier(text)

Device set to use cuda:0


[{'entity': 'B-location',
  'score': np.float32(0.49121246),
  'index': 2,
  'word': 'golden',
  'start': 4,
  'end': 10},
 {'entity': 'B-location',
  'score': np.float32(0.27243415),
  'index': 3,
  'word': 'state',
  'start': 11,
  'end': 16},
 {'entity': 'B-group',
  'score': np.float32(0.21433908),
  'index': 4,
  'word': 'warriors',
  'start': 17,
  'end': 25},
 {'entity': 'B-location',
  'score': np.float32(0.6681575),
  'index': 13,
  'word': 'san',
  'start': 80,
  'end': 83},
 {'entity': 'B-location',
  'score': np.float32(0.5835237),
  'index': 14,
  'word': 'francisco',
  'start': 84,
  'end': 93}]